# Adding Memory

In [1]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [11]:
memory = ConversationBufferMemory(return_messages=True)

In [12]:
memory.load_memory_variables({})

{'history': []}

In [13]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [15]:
inputs = {"input": "Hey! I'm Sohan"}
response = chain.invoke(inputs)
response.content

'Hi Sohan! How can I assist you today?'

In [16]:
memory.save_context(inputs, {"output": response.content})

In [17]:
memory.load_memory_variables({})

{'history': [HumanMessage(content="Hey! I'm Sohan"),
  AIMessage(content='Hi Sohan! How can I assist you today?')]}

In [18]:
inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
response

AIMessage(content='Your name is Sohan. Is there anything else I can help you with?')